In [1]:
# ========== ----- ========== Import Libraries ========== ----- ========== #
!pip install pyspark

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from joblib import dump, load

from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Import required libraries
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import LogisticRegressionWithSGD
from pyspark import SparkContext

# Import necessary libraries
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col
from pyspark.sql import SparkSession

# ========== ----- ========== End ========== ----- ========== #

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
##The dataset is loaded to your GDrive so need to be mounted
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
# ========== ----- ========== Import Dataset ========== ----- ========== #

# Create a SparkContext object
spark=SparkSession.builder\
    .master("local[5]")\
    .appName("LogisticRegression")\
    .getOrCreate()

# Load data into Spark DataFrame
df = spark.read.format("csv").option(
    "header", "true").load("/content/drive/MyDrive/final_df.csv")

header = df.first()
print(header)

# ========== ----- ========== End ========== ----- ========== #


Row(client_catg='11', avg_elec_reading_remarque='6.971428571', avg_elec_consommation_level_1='352.4', avg_elec_consommation_level_2='10.57142857', avg_elec_consommation_level_3='0', avg_elec_consommation_level_4='0', avg_elec_months_number='4.628571429', avg_gaz_reading_remarque='0', avg_gaz_consommation_level_1='0', avg_gaz_consommation_level_2='0', avg_gaz_consommation_level_3='0', avg_gaz_consommation_level_4='0', avg_gaz_months_number='0', min_elec_reading_remarque='6', min_elec_consommation_level_1='38', min_elec_consommation_level_2='0', min_elec_consommation_level_3='0', min_elec_consommation_level_4='0', min_elec_months_number='2', min_gaz_reading_remarque='0', min_gaz_consommation_level_1='0', min_gaz_consommation_level_2='0', min_gaz_consommation_level_3='0', min_gaz_consommation_level_4='0', min_gaz_months_number='0', max_elec_reading_remarque='9', max_elec_consommation_level_1='1200', max_elec_consommation_level_2='186', max_elec_consommation_level_3='0', max_elec_consommat

In [16]:
# ========== ----- ========== Prepare The Data ========== ----- ========== #

# Perform some data preprocessing using the map function
def preprocess(row):
    # Convert string fields to numeric fields
    row = [float(x) if x.isdigit() else x for x in row]
    # Handle missing values
    row = [0.0 if x is None else x for x in row]
    return row


df = df.rdd.map(preprocess).toDF(df.columns)

df = df.withColumn("avg_elec_reading_remarque", df["avg_elec_reading_remarque"].cast("double"))\
      .withColumn("avg_elec_consommation_level_1", df["avg_elec_consommation_level_1"].cast("double"))\
      .withColumn("avg_elec_consommation_level_2", df["avg_elec_consommation_level_2"].cast("double"))\
      .withColumn("avg_elec_months_number", df["avg_elec_months_number"].cast("double"))\
      .withColumn("std_elec_reading_remarque", df["std_elec_reading_remarque"].cast("double"))\
      .withColumn("std_elec_consommation_level_1", df["std_elec_consommation_level_1"].cast("double"))\
      .withColumn("std_elec_consommation_level_2", df["std_elec_consommation_level_2"].cast("double"))\
      .withColumn("std_elec_months_number", df["std_elec_months_number"].cast("double"))\

# ========== ----- ========== End ========== ----- ========== #


In [17]:
# ========== ----- ========== Logistic Regression Model ========== ----- ========== #

# Define input features and target variable
input_cols = df.columns[:-1]
assembler = VectorAssembler(inputCols=input_cols, outputCol='features',handleInvalid="skip")
data = assembler.transform(df).select(col("features"), col("target").alias("label")).na.drop()

# Split data into training and testing sets
(training_data, testing_data) = data.randomSplit([0.7, 0.3])

# Build the logistic regression model
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0)
model = lr.fit(training_data)

# Make predictions on the testing set
predictions = model.transform(testing_data)

# Evaluate the model
evaluator = BinaryClassificationEvaluator()
print('Area Under ROC Curve:', evaluator.evaluate(predictions))

# ========== ----- ========== End ========== ----- ========== #

Area Under ROC Curve: 0.6866667937364653


In [14]:
# ========== ----- ========== Spark Session ========== ----- ========== #

# Stop the Spark session
spark.stop()

# ========== ----- ========== End ========== ----- ========== #